In [1]:
from sentence_transformers import SentenceTransformer

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
model_name = "multi-qa-distilbert-cos-v1"
model = SentenceTransformer(model_name)
def get_embeddings(input):
    return model.encode(input)

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Q1

In [3]:
user_question = "I just discovered the course. Can I still join it?"
get_embeddings(user_question)[0]

np.float32(0.078222655)

### Q2

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [5]:
import numpy as np
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    temp_embed = get_embeddings(qa_text)
    embeddings.append(temp_embed)

X = np.array(embeddings)
X.shape

100%|██████████| 948/948 [02:54<00:00,  5.44it/s]


(948, 768)

### Q3

In [6]:
v = get_embeddings(user_question)
scores = X.dot(v)
max(scores)

np.float32(0.6506573)

### Q4

In [7]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

In [8]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [9]:
len(ground_truth)

1830

In [19]:
import json
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [28]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
relevance_list = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    temp_embed = get_embeddings(q['question'])
    results = search_engine.search(temp_embed, num_results=5)
    # [print(d['id'], d['question']) for d in results]
    d = [res['id'] == doc_id for res in results]
    relevance_list.append(d)
    # print("_____________\n")

100%|██████████| 1830/1830 [01:35<00:00, 19.11it/s]


In [30]:
cnt = 0
for relevance in relevance_list:
    cnt += int(sum(relevance) > 0)

cnt/ len(ground_truth)

0.9218579234972678

### Q5

In [34]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://localhost:9200")
print(client.info())

{'name': '8cc54760ddb6', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8HSErMRuQNeevzt9cyhEjg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [44]:
settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type":"keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "ground_truth"
client.indices.delete(index=index_name)
client.indices.create(index=index_name, body=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ground_truth'})

In [45]:
for doc in tqdm(ground_truth):
    doc['question_vector'] = get_embeddings(input=doc['question'])
    client.index(index=index_name, document=doc)

100%|██████████| 1830/1830 [01:58<00:00, 15.40it/s]


In [50]:
def search_query(query):
    vq = get_embeddings(query)
    search_query = {
        "knn": {
                "field": "question_vector",
                "k": 5,
                "query_vector": vq,
                "num_candidates": 10000,
        },
        "_source": ["question", "course", "document"]
    }
    results = client.search(index= index_name, body=search_query)
    return results['hits']['hits']

search_query(user_question)[0]['_source']['document']

'ee58a693'

### Q6

In [52]:
rel_list = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_query(q['question'])
    d = [res['_source']['document'] == doc_id for res in results]
    rel_list.append(d)


100%|██████████| 1830/1830 [01:30<00:00, 20.24it/s]


In [55]:
cnt = 0
for relevance in rel_list:
    if True in relevance:
        cnt += 1

cnt/len(rel_list)

0.9972677595628415